# Example to get energies (total, electronic, nuclear, heat of formation, orbital), charges, dipole, forces, density matrix.

In [7]:
import sys
### path to PYSEQM ###
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
#device = torch.device('cpu')

In [8]:
%%time

### create molecule object:
species = torch.as_tensor([[8,6,1,1],
                           [8,6,1,1],
                           [8,8,6,0]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.23,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [0.0,0.0,0.0]            # zero-padding for batching
                              ]
                            ], device=device)

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [0,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True, # store orbital energies
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on molecules:
esdriver(molecules)

/vast/home/vishikh/projects/PYSEQM/seqm/seqm_functions/rcis_batch.py:217: SyntaxWarning: invalid escape sequence '\s'
  """


ValueError: All molecules in the batch should be of the same type with same number of orbitals and electrons

In [9]:
molecules.nocc

tensor([6, 6, 8], device='cuda:0')

In [10]:
print(' Total Energy (eV):\n', molecules.Etot)
print('\n Electronic Energy (eV): ', molecules.Eelec)
print('\n Nuclear Energy (eV):\n', molecules.Enuc)
print('\n Heat of Formation (ev):\n', molecules.Hf)
print('\n HOMO-LUMO gap (eV):\n', molecules.e_gap)
print('\n Orbital energies (eV):\n', molecules.e_mo)
print('\n Charges:\n', molecules.q)
print('\n Dipole:\n', molecules.d)
print('\n Forces (eV/A):\n', molecules.force)

 Total Energy (eV):
 None

 Electronic Energy (eV):  None

 Nuclear Energy (eV):
 None

 Heat of Formation (ev):
 None

 HOMO-LUMO gap (eV):
 None

 Orbital energies (eV):
 None

 Charges:
 None

 Dipole:
 None

 Forces (eV/A):
 None


In [11]:
print('\n Density matrix:\n', molecules.dm)


 Density matrix:
 None


In [12]:
molecules.coordinates

Parameter containing:
tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 1.2200,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 1.8200, -0.9400,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 1.2200,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 1.8200, -0.9400,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 1.2300,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]], device='cuda:0', requires_grad=True)